In [2]:
import pandas as pd

data = pd.read_csv("unclean.csv")

/var/tmp/ipykernel_10508/752483283.py:3: DtypeWarning: Columns (249,250,255,256,257,258,259,264,265,266,267,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,308,309,315,316,317,318,319,321,325,326,327,338,339,344,345,346,347,348,349,350,351,354,355,356,357,368,369,372,373,374,377,385,386,387,388,389,390,391,393,394,2060,2061) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("unclean.csv")


In [3]:
data.shape

(3143, 2066)

In [4]:
data["Code"].isnull().sum()

0

In [5]:
# All zeros which is good
data["Latitude"].isnull().sum()
data["Expected Population 2020, total, ages 10-14"].isnull().sum()

0

In [6]:
duplicates = data.columns[data.T.duplicated(keep = False)].tolist()
len(duplicates)

171

In [7]:
def remove_duplicates(df):
    duplicate_mask = df.T.duplicated(keep = 'first')
    return df.loc[:, ~duplicate_mask]

data = remove_duplicates(data)

In [8]:
data.shape

(3143, 1897)

In [9]:
def remove_highNA(df, threshold = 0.3):
    percent = df.isnull().mean()
    keep = percent[percent <= threshold].index
    return df[keep]

data = remove_highNA(data)
data.shape

(3143, 1394)

In [10]:
import pandas as pd
import numpy as np  # Import numpy directly

def remove_highly_correlated_columns_numeric(df, threshold=1.0):
    """
    Removes one of each pair of highly correlated columns, applied only to numeric columns.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        threshold (float): Correlation threshold (default is 0.9).
    
    Returns:
        pd.DataFrame: DataFrame with highly correlated numeric columns removed.
    """
    # Select only numeric columns
    numeric_df = df.select_dtypes(include='number')
    
    # Compute the correlation matrix
    corr_matrix = numeric_df.corr().abs()
    
    # Select the upper triangle of the correlation matrix
    upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)  # Use numpy directly
    )
    
    # Find columns with correlation greater than the threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    # Drop those columns from the numeric DataFrame
    numeric_df_cleaned = numeric_df.drop(columns=to_drop)
    
    # Combine numeric and non-numeric columns
    non_numeric_df = df.select_dtypes(exclude='number')
    df_cleaned = pd.concat([numeric_df_cleaned, non_numeric_df], axis=1)
    
    return df_cleaned

data = remove_highly_correlated_columns_numeric(data, 1.0)


In [11]:
data.shape

(3143, 1390)

In [12]:
import pandas as pd
import numpy as np
import re

def remove_highly_correlated_ignore_digit_columns(df, threshold=0.8):
    columns_without_digits = [col for col in df.columns if not re.search(r'\d', col)]
    numeric_df = df[columns_without_digits].select_dtypes(include='number')
    corr_matrix = numeric_df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return df.drop(columns=to_drop)

new_data = remove_highly_correlated_ignore_digit_columns(data)
new_data.shape
data = new_data


In [13]:
data["Net Migration Rate 2010s, total, ages 40-44"].isnull().sum()

0

In [14]:

def print_nonnumeric_column_names(df):
    nonnumeric_columns = df.select_dtypes(exclude='number').columns.tolist()
    print("Non-numeric column names:")
    for col in nonnumeric_columns:
        print(col)
print_nonnumeric_column_names(data)

Non-numeric column names:
statecounty_x
Populations_Vulnerable_to_Predicted_Surface_Smoke_from_Wildland_Fires_NOAA2020
White_Single_Race_Male_Percent_of_Diabetes_In_Total_Deaths_CDC2018_22_x
White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22
White_Single_Race_Female_Percent_of_Hypertension_In_Total_Deaths_CDC2018_22_x
White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22
Number_of_hospitals_County_CDCnehTracking_2020
Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20
mapped_match
stname
name
State
County [2]
County Seat(s) [3]
Water Area km²
Water Area mi²
Latitude
Longitude


In [15]:
data = data.drop(columns = ['mapped_match', 'name', 'County [2]', 'County Seat(s) [3]','State'])

In [16]:
data.shape

(3143, 1320)

In [17]:
data.drop_duplicates()

,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,Total_population_16yr_And_Over_For_Work_Commute_ACS2017_21,...,"White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22",White_Single_Race_Female_Percent_of_Hypertension_In_Total_Deaths_CDC2018_22_x,"White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22",Number_of_hospitals_County_CDCnehTracking_2020,Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20,stname,Water Area km²,Water Area mi²,Latitude,Longitude
0,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,...,52.7,0.0%,35.5,1,11.9,Alabama,25.776,9.952,+32.536382°,–86.644490°
1,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,...,34.1,0.0%,42.6,4,15.0,Alabama,1133.19,437.527,+30.659218°,–87.746067°
2,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,...,NaN,NaN,NaN,1,15.9,Alabama,50.865,19.639,+31.870670°,–85.405456°
3,0.01,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,...,NaN,0.0%,49.4,1,18.0,Alabama,9.289,3.587,+33.015893°,–87.127148°
4,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,...,21.0,0.0%,53.6,1,18.6,Alabama,15.157,5.852,+33.977448°,–86.567246°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.00,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,...,50.0,0.0%,30.9,2,11.9,Wyoming,166.887,64.436,+41.660339°,–108.875676°
3139,0.00,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,...,NaN,NaN,NaN,1,Data Not Collected,Wyoming,572.266,220.953,+44.049321°,–110.588102°
3140,0.00,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,...,Unreliable,0.0%,Unreliable,2,Data Not Collected,Wyoming,16.342,6.31,+41.284726°,–110.558947°
3141,0.00,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,...,NaN,NaN,NaN,1,Data Not Collected,Wyoming,10.762,4.155,+43.878831°,–107.669052°


In [18]:
def clean_object_columns(df):
    # Identify object columns
    object_columns = df.select_dtypes(include='object').columns

    def convert_value(value):
        try:
            if isinstance(value, str):
                value = value.strip()  # Remove leading/trailing spaces
                value = value.replace(',', '').replace('°', '')  # Remove commas, degree symbols
                value = value.replace('–', '-')  # Handle en-dash as minus sign
                if value.endswith('%'):  # Convert percentages
                    return float(value.strip('%')) / 100
                return float(value)  # Attempt to convert to float
        except ValueError:
            return value  # Return original value if conversion fails
        return value  # Return original value if not a string

    # Apply conversion only to object columns
    for col in object_columns:
        df[col] = df[col].apply(convert_value)

    return df



clean_object_columns(data)

,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,Total_population_16yr_And_Over_For_Work_Commute_ACS2017_21,...,"White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22",White_Single_Race_Female_Percent_of_Hypertension_In_Total_Deaths_CDC2018_22_x,"White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22",Number_of_hospitals_County_CDCnehTracking_2020,Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20,stname,Water Area km²,Water Area mi²,Latitude,Longitude
0,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,...,52.7,0.0,35.5,1.0,11.9,Alabama,25.776,9.952,32.536382,-86.644490
1,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,...,34.1,0.0,42.6,4.0,15.0,Alabama,1133.19,437.527,30.659218,-87.746067
2,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,...,NaN,NaN,NaN,1.0,15.9,Alabama,50.865,19.639,31.870670,-85.405456
3,0.01,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,...,NaN,0.0,49.4,1.0,18.0,Alabama,9.289,3.587,33.015893,-87.127148
4,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,...,21.0,0.0,53.6,1.0,18.6,Alabama,15.157,5.852,33.977448,-86.567246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.00,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,...,50.0,0.0,30.9,2.0,11.9,Wyoming,166.887,64.436,41.660339,-108.875676
3139,0.00,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,...,NaN,NaN,NaN,1.0,Data Not Collected,Wyoming,572.266,220.953,44.049321,-110.588102
3140,0.00,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,...,Unreliable,0.0,Unreliable,2.0,Data Not Collected,Wyoming,16.342,6.31,41.284726,-110.558947
3141,0.00,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,...,NaN,NaN,NaN,1.0,Data Not Collected,Wyoming,10.762,4.155,43.878831,-107.669052


In [19]:
def print_unique_non_numeric_entries(df):
    unique_non_numeric = set()
    for value in df.applymap(lambda x: x if not isinstance(x, (int, float)) else None).stack().values:
        if value is not None and value not in unique_non_numeric:
            unique_non_numeric.add(value)
            print(value)


In [20]:
print(data.dtypes)


AirPollutant: 1,3-butadiene_ug/m^3_EPA2018    float64
AirPollutant: Acetaldehyde_ug/m^3_EPA2017     float64
AirPollutant: Acetaldehyde_ug/m^3_EPA2018     float64
AirPollutant: Benzene_ug/m^3_EPA2017          float64
AirPollutant: Benzene_ug/m^3_EPA2018          float64
                                               ...   
stname                                         object
Water Area km²                                 object
Water Area mi²                                 object
Latitude                                      float64
Longitude                                     float64
Length: 1320, dtype: object


In [21]:
print_nonnumeric_column_names(data)

Non-numeric column names:
statecounty_x
White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22
White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22
Number_of_hospitals_County_CDCnehTracking_2020
Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20
stname
Water Area km²
Water Area mi²


In [22]:

def find_unique_non_numeric_examples(df, columns):
    """
    Identifies unique non-numeric examples from specified columns in a DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to check for non-numeric examples.

    Returns:
        dict: A dictionary with column names as keys and sets of unique non-numeric values as values.
    """
    non_numeric_examples = {}
    for column in columns:
        if column in df.columns:  # Check if the column exists in the DataFrame
            non_numeric_values = set(
                df[column].apply(lambda x: x if not isinstance(x, (int, float)) else None).dropna()
            )
            non_numeric_examples[column] = non_numeric_values
    return non_numeric_examples

examples = find_unique_non_numeric_examples(data, [
    'White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Number_of_hospitals_County_CDCnehTracking_2020',
    'Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20',
    'Water Area km²',
    'Water Area mi²'
]
)
for column, examples in examples.items():
    print(f"Non-numeric examples from column '{column}':")
    for example in examples:
        print(example)
print('-' * 50)

Non-numeric examples from column 'White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22':
Not Available
Unreliable
Non-numeric examples from column 'White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22':
Not Available
Unreliable
Non-numeric examples from column 'Number_of_hospitals_County_CDCnehTracking_2020':
No Counts
Non-numeric examples from column 'Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20':
Data Not Collected
Non-numeric examples from column 'Water Area km²':
-
Non-numeric examples from column 'Water Area mi²':
-
--------------------------------------------------


In [23]:
columns_to_process = [
    'White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Number_of_hospitals_County_CDCnehTracking_2020',
    'Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20',
    'Water Area km²',
    'Water Area mi²'
]

non_numeric_replacements = {
    'Unreliable': np.nan,
    'Not Available': np.nan,
    'No Counts': np.nan,
    'Data Not Collected': np.nan,
    '-': np.nan
}

def change_to_nan(df):
    for column in columns_to_process:
        if column in df.columns:
            df[column] = df[column].replace(non_numeric_replacements)

change_to_nan(data)

examples = find_unique_non_numeric_examples(data, [
    'White_Single_Race_Male_Diabetes_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'White_Single_Race_Female_Hypertension_Deaths_Rate_Per_100,000_Residents_CDC2018_22',
    'Number_of_hospitals_County_CDCnehTracking_2020',
    'Number_of_Pharmacies_available_per100000_CDCnehTracking_2016_20',
    'Water Area km²',
    'Water Area mi²'
]
)


In [24]:
print_nonnumeric_column_names(data)

Non-numeric column names:
statecounty_x
stname


In [25]:
remove_highNA(data, 0.3)

,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,Total_population_16yr_And_Over_For_Work_Commute_ACS2017_21,...,statecounty_x,Populations_Vulnerable_to_Predicted_Surface_Smoke_from_Wildland_Fires_NOAA2020,White_Single_Race_Male_Percent_of_Diabetes_In_Total_Deaths_CDC2018_22_x,White_Single_Race_Female_Percent_of_Hypertension_In_Total_Deaths_CDC2018_22_x,Number_of_hospitals_County_CDCnehTracking_2020,stname,Water Area km²,Water Area mi²,Latitude,Longitude
0,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,...,01autauga,59759.0,0.0,0.0,1.0,Alabama,25.776,9.952,32.536382,-86.644490
1,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,...,01baldwin,246435.0,0.0,0.0,4.0,Alabama,1133.190,437.527,30.659218,-87.746067
2,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,...,01barbour,24706.0,NaN,NaN,1.0,Alabama,50.865,19.639,31.870670,-85.405456
3,0.01,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,...,01bibb,22005.0,NaN,0.0,1.0,Alabama,9.289,3.587,33.015893,-87.127148
4,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,...,01blount,59512.0,0.0,0.0,1.0,Alabama,15.157,5.852,33.977448,-86.567246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.00,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,...,56sweetwater,41345.0,0.0,0.0,2.0,Wyoming,166.887,64.436,41.660339,-108.875676
3139,0.00,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,...,56teton,23287.0,NaN,NaN,1.0,Wyoming,572.266,220.953,44.049321,-110.588102
3140,0.00,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,...,56uinta,20712.0,0.0,0.0,2.0,Wyoming,16.342,6.310,41.284726,-110.558947
3141,0.00,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,...,56washakie,7719.0,NaN,NaN,1.0,Wyoming,10.762,4.155,43.878831,-107.669052


In [26]:
remove_highNA(data, 0.2)

,"AirPollutant: 1,3-butadiene_ug/m^3_EPA2018",AirPollutant: Acetaldehyde_ug/m^3_EPA2017,AirPollutant: Acetaldehyde_ug/m^3_EPA2018,AirPollutant: Benzene_ug/m^3_EPA2017,AirPollutant: Benzene_ug/m^3_EPA2018,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2017,AirPollutant: Carbon tetrachloride_ug/m^3_EPA2018,AirPollutant: Formaldehyde_ug/m^3_EPA2017,AirPollutant: Formaldehyde_ug/m^3_EPA2018,Total_population_16yr_And_Over_For_Work_Commute_ACS2017_21,...,Sort [1],Population (2010),Land Area km²,statecounty_x,Populations_Vulnerable_to_Predicted_Surface_Smoke_from_Wildland_Fires_NOAA2020,stname,Water Area km²,Water Area mi²,Latitude,Longitude
0,0.01,1.50,1.41,0.19,0.18,0.51,0.37,1.84,1.69,25204.0,...,1,54571,1539.582,01autauga,59759.0,Alabama,25.776,9.952,32.536382,-86.644490
1,0.01,1.15,1.10,0.17,0.16,0.51,0.37,1.47,1.40,94637.0,...,2,182265,4117.522,01baldwin,246435.0,Alabama,1133.190,437.527,30.659218,-87.746067
2,0.02,1.51,1.39,0.18,0.17,0.50,0.37,1.83,1.64,8324.0,...,3,27457,2291.819,01barbour,24706.0,Alabama,50.865,19.639,31.870670,-85.405456
3,0.01,1.61,1.54,0.18,0.18,0.51,0.37,1.90,1.86,7626.0,...,4,22915,1612.481,01bibb,22005.0,Alabama,9.289,3.587,33.015893,-87.127148
4,0.01,1.24,1.19,0.20,0.19,0.50,0.37,1.65,1.59,23240.0,...,5,57322,1669.962,01blount,59512.0,Alabama,15.157,5.852,33.977448,-86.567246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,0.00,0.32,0.29,0.10,0.10,0.42,0.31,0.51,0.49,20061.0,...,3139,43806,27004.897,56sweetwater,41345.0,Wyoming,166.887,64.436,41.660339,-108.875676
3139,0.00,0.37,0.30,0.07,0.07,0.41,0.30,0.45,0.42,12795.0,...,3140,21294,10347.984,56teton,23287.0,Wyoming,572.266,220.953,44.049321,-110.588102
3140,0.00,0.30,0.27,0.08,0.08,0.41,0.30,0.50,0.48,9236.0,...,3141,21118,5390.450,56uinta,20712.0,Wyoming,16.342,6.310,41.284726,-110.558947
3141,0.00,0.41,0.34,0.09,0.10,0.45,0.33,0.54,0.50,3715.0,...,3142,8533,5797.815,56washakie,7719.0,Wyoming,10.762,4.155,43.878831,-107.669052


In [27]:
data.to_csv("cleaned version1: incomplete imputation.csv")